In [10]:
####株価データ統合####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import itertools
import os
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
import re
import sys

#np.random.seed(98537)

In [6]:
##データの読み込み
#読み込むデータの一覧
f = open("D:/Statistics/data/stock_price/st-jp/f-list.txt")
code_name = f.read()  # ファイル終端まで全て読んだデータを返す
code_name = str.split(code_name, "\n")[1:] 
code = [re.sub(".CSV", "", code_name[i]) for i in range(len(code_name))]

In [7]:
#証券コードごとにデータを読み込み
n = len(code_name)-1
input_path = "D:/Statistics/data/stock_price/st-jp/dd/{code_name}"
temporal_data = [i for i in range(n)]
id_list = [i for i in range(n)]
code_list = [i for i in range(n)]
for i in range(n):
    temporal_data[i] = np.array(pd.read_csv(input_path.format(code_name=code_name[i]), header=None))
    code_list[i] = np.repeat(code[i], temporal_data[i].shape[0])
    id_list[i] = np.repeat(i, temporal_data[i].shape[0])

In [14]:
##業界ごとに株価データを書き出し
#新しいディレクトリを作成
company_info = pd.read_csv("D:/Statistics/data/stock_price/st-jp/company_info.csv")[["コード", "銘柄名", "業種"]]
industry = np.unique(company_info["業種"])
no = np.append(np.array(["01", "02", "03", "04", "05", "06","07", "08", "09"]),
               np.array(np.arange(10, industry.shape[0]+1), "U2"))
industry_no = no + (np.array(np.repeat("_", no.shape[0]), "U2") + industry)
company_info_df = pd.merge(company_info, pd.DataFrame({"業種": industry, "業種no": industry_no}), on="業種", how="left")
company_info = np.array(company_info_df)

new_dir_path = "D:/Statistics/data/stock_price/individual/{new_dir}"
for j in range(industry_no.shape[0]):
    if os.path.exists(new_dir_path.format(new_dir=industry_no[j]))==True:
        next
    else:
        os.mkdir(new_dir_path.format(new_dir=industry_no[j]))

#企業ごとにに株価データを出力
n = len(temporal_data)
index = np.array([0, 4])   #日付と終値のみ取得
for i in range(n):
    temporal_data0 = temporal_data[i][:, index]
    temp = company_info[company_info[:, 0]==np.array(code_list[i][0], dtype="int"), ].reshape(-1)
    code0 = temp[0]; company0 = temp[1]; industry0 = temp[3] 
    company0 = company0.replace("/", "_")   #「/」を置き換える
    out_df = pd.DataFrame({"company_ticker": code0, "date": temporal_data0[:, 0], "stock_price": temporal_data0[:, 1]})
    out_path = "D:/Statistics/data/stock_price/individual/{industry}/{code}_{company}.csv"
    out_df.to_csv(out_path.format(industry=industry0, code=code0, company=company0), index=False, encoding="utf-8")

In [15]:
##業界コードを出力
industry_vec = np.array(company_info_df["業種no"])
for j in range(industry_no.shape[0]):
    index = np.where(industry_vec==industry_no[j])[0]
    tmp = company_info_df.iloc[index, ]
    tmp.index = np.arange(index.shape[0])
    joint_df = pd.concat([pd.DataFrame({"industry_code": np.repeat(industry_no[j][0:2], index.shape[0]), 
                                        "industry_name": np.repeat(industry_no[j][3:], index.shape[0])}), 
                          tmp[["コード", "銘柄名"]]], axis=1)
    joint_df.columns = ["industry_code", "industry_name", "company_ticker", "company_name"]

    out_path = "D:/Statistics/data/stock_price/individual/{industry_no}.csv"
    joint_df.to_csv(out_path.format(industry_no=industry_no[j]), index=False, encoding="utf-8")

In [16]:
##すべてのデータをパネルデータとして書き出す
#リストに変換
pannel_data = np.array(list(itertools.chain(*[temporal_data[i] for i in range(n)])))
code = np.array(list(itertools.chain(*[code_list[i] for i in range(n)])))
r_id = np.array(list(itertools.chain(*[id_list[i] for i in range(n)])))
del temporal_data, code_list, id_list

In [17]:
#データフレームに加工
pannel_df = pd.DataFrame(pannel_data)
pannel_df.columns = ["日付", "始値", "高値", "安値", "終値", "出来高", "信用売残", "信用買銭"]
reference_id = pd.DataFrame(r_id)
reference_id.columns = ["reference_id"]

In [18]:
#企業情報を結合
company_info = pd.read_csv("D:/Statistics/data/stock_price/st-jp/company_info.csv")
company_info["銘柄名"] = company_info["銘柄名"].str.replace("建設業", "建設")
company_panel = company_info[["コード", "銘柄名", "業種"]].iloc[r_id, ]
company_panel.index = np.arange(company_panel.shape[0])
joint_data = pd.concat([reference_id, company_panel, pannel_df], axis=1)

In [19]:
#データの書き出し
out1 = "D:/Statistics/data/stock_price/stock_panel_data.csv"
out2 = "D:/Statistics/data/stock_price/st-jp/company_info.csv"
joint_data.to_csv(out1, sep=",")
company_info.to_csv(out2, sep=",")
joint_data

,reference_id,コード,銘柄名,業種,日付,始値,高値,安値,終値,出来高,信用売残,信用買銭
0,0,1301,極洋,水産・農林,2001/07/13,1258,1258,1239,1241,706400,118000,55000
1,0,1301,極洋,水産・農林,2001/07/16,1245,1249,1237,1241,50200,118000,55000
2,0,1301,極洋,水産・農林,2001/07/17,1225,1229,1220,1224,78700,118000,55000
3,0,1301,極洋,水産・農林,2001/07/18,1226,1226,1197,1199,113100,118000,55000
4,0,1301,極洋,水産・農林,2001/07/19,1195,1200,1189,1193,93400,118000,55000
5,0,1301,極洋,水産・農林,2001/07/23,1196,1196,1158,1159,96900,37000,64000
6,0,1301,極洋,水産・農林,2001/07/24,1157,1174,1155,1174,99700,37000,64000
7,0,1301,極洋,水産・農林,2001/07/25,1174,1198,1172,1188,76300,37000,64000
8,0,1301,極洋,水産・農林,2001/07/26,1192,1195,1187,1192,17000,37000,64000
9,0,1301,極洋,水産・農林,2001/07/27,1192,1194,1179,1185,33200,12000,69000
